In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import mesa

# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

# Cache results without solara; not on grid

In [3]:
from cacheable_model import CacheableModel
import cacheable_model

In [4]:
import shutil

def delete_directory(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory {directory_path} and its contents have been deleted.")

try:
    directory = "test_path"
    delete_directory(directory)
except:
    pass

Directory test_path and its contents have been deleted.


In [5]:
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates.pop(
            cellmates.index(self)
        )  # Ensure agent is not giving money to itself
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            if other == self:
                print("I JUST GAVE MONEY TO MYSELF HEHEHE!")

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [6]:
model = MoneyModel(100, 10, 10)

In [34]:
model.__dict__

{'_seed': 0.5967757455124764,
 'random': <random.Random at 0x5613fbf8a970>,
 '_steps': 1000,
 '_time': 1000,
 'running': True,
 'schedule': <mesa.time.RandomActivation at 0x7f77e00c4370>,
 'current_id': 0,
 'agents_': defaultdict(dict,
             {__main__.MoneyAgent: {<__main__.MoneyAgent at 0x7f77e00c45b0>: None,
               <__main__.MoneyAgent at 0x7f77e00c49a0>: None,
               <__main__.MoneyAgent at 0x7f77e00c48e0>: None,
               <__main__.MoneyAgent at 0x7f77e00c4910>: None,
               <__main__.MoneyAgent at 0x7f77e00c7df0>: None,
               <__main__.MoneyAgent at 0x7f77e00c4a30>: None,
               <__main__.MoneyAgent at 0x7f77e00c4b20>: None,
               <__main__.MoneyAgent at 0x7f77e00c4b50>: None,
               <__main__.MoneyAgent at 0x7f77e00c4ca0>: None,
               <__main__.MoneyAgent at 0x7f77e00c4cd0>: None,
               <__main__.MoneyAgent at 0x7f77e00c4dc0>: None,
               <__main__.MoneyAgent at 0x7f77e00c4d30>: None,

In [39]:
model.agents_
for i in model.agents_.keys():
    print(i)

defaultdict(dict,
            {__main__.MoneyAgent: {<__main__.MoneyAgent at 0x7f77e00c45b0>: None,
              <__main__.MoneyAgent at 0x7f77e00c49a0>: None,
              <__main__.MoneyAgent at 0x7f77e00c48e0>: None,
              <__main__.MoneyAgent at 0x7f77e00c4910>: None,
              <__main__.MoneyAgent at 0x7f77e00c7df0>: None,
              <__main__.MoneyAgent at 0x7f77e00c4a30>: None,
              <__main__.MoneyAgent at 0x7f77e00c4b20>: None,
              <__main__.MoneyAgent at 0x7f77e00c4b50>: None,
              <__main__.MoneyAgent at 0x7f77e00c4ca0>: None,
              <__main__.MoneyAgent at 0x7f77e00c4cd0>: None,
              <__main__.MoneyAgent at 0x7f77e00c4dc0>: None,
              <__main__.MoneyAgent at 0x7f77e00c4d30>: None,
              <__main__.MoneyAgent at 0x7f77e00c4e20>: None,
              <__main__.MoneyAgent at 0x7f77e00c4e50>: None,
              <__main__.MoneyAgent at 0x7f77e00c4be0>: None,
              <__main__.MoneyAgent at 0x7f77e0

<class '__main__.MoneyAgent'>
<class 'mesa.agent.Agent'>


In [7]:
# condition to cache the results specifically
def condition_function(model_vars):
        return model_vars.get('Gini', 0)[-1] > 0.7

In [8]:
number_of_steps = 1000
cacheable_model = CacheableModel(model, directory, number_of_steps, condition_function)

In [9]:
for i in range(number_of_steps):
    print(i)
    cacheable_model.model.step()
    cacheable_model.cache()

0
Condition not met at step 1. No data to save.
1
Condition not met at step 2. No data to save.
2
Condition not met at step 3. No data to save.
3
Condition not met at step 4. No data to save.
4
Condition not met at step 5. No data to save.
5
Condition not met at step 6. No data to save.
6
Condition not met at step 7. No data to save.
7
Condition not met at step 8. No data to save.
8
Condition not met at step 9. No data to save.
9
Condition not met at step 10. No data to save.
10
Condition not met at step 11. No data to save.
11
Condition not met at step 12. No data to save.
12
Condition not met at step 13. No data to save.
13
Condition not met at step 14. No data to save.
14
Condition not met at step 15. No data to save.
15
Condition not met at step 16. No data to save.
16
Condition not met at step 17. No data to save.
17
Condition not met at step 18. No data to save.
18
Condition not met at step 19. No data to save.
19
Condition not met at step 20. No data to save.
20
Condition not me

In [10]:
model_df, agent_df = cacheable_model.combine_dataframes()
model_df
agent_df

,Gini
0,0.0000
1,0.2602
2,0.4262
3,0.4538
4,0.4824
...,...
995,0.6944
996,0.6878
997,0.6712
998,0.6660


Wealth
Step AgentID        
0    0             1
     1             1
     2             1
     3             1
     4             1
...              ...
999  95            6
     96            1
     97            1
     98            6
     99            0

[100000 rows x 1 columns]

In [11]:
import pyarrow.parquet as pq
import pandas as pd
import glob

# Get a list of all Parquet files
model_files = glob.glob(f'{directory}/model_data_*.parquet')
agent_files = glob.glob(f'{directory}/agent_data_*.parquet')
special_files = glob.glob(f'{directory}/special_results.parquet')

# Initialize lists to hold dataframes
model_dfs = []
agent_dfs = []
special_dfs = []

# Read and append each file to the list
for model_file in model_files:
    table = pq.read_table(model_file)
    df = table.to_pandas()
    model_dfs.append(df)

for agent_file in agent_files:
    table = pq.read_table(agent_file)
    df = table.to_pandas()
    agent_dfs.append(df)

for special_file in special_files:
    table = pq.read_table(special_file)
    df = table.to_pandas()
    special_dfs.append(df)

# Concatenate all DataFrames
model_df = pd.concat(model_dfs, ignore_index=True)
agent_df = pd.concat(agent_dfs)
special_df = pd.concat(special_dfs).set_index("Step")

# Display the combined DataFrames
model_df
agent_df

,Gini
0,0.0000
1,0.2602
2,0.4262
3,0.4538
4,0.4824
...,...
995,0.6944
996,0.6878
997,0.6712
998,0.6660


Wealth
Step AgentID        
0    0             1
     1             1
     2             1
     3             1
     4             1
...              ...
999  25            1
     70            0
     30            1
     63            0
     56            1

[100000 rows x 1 columns]

In [12]:
special_df

,Gini
Step,
95,0.7018
105,0.7040
106,0.7268
107,0.7086
108,0.7004
...,...
948,0.7176
949,0.7094
950,0.7156


# caching for solar & grid visualisation

In [13]:
import networkx as nx
import solara
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator

In [14]:
def _draw_grid(space, space_ax, agent_portrayal):
    def portray(g):
        x = []
        y = []
        s = []  # size
        c = []  # color
        for i in range(g.width):
            for j in range(g.height):
                content = g._grid[i][j]
                if not content:
                    continue
                if not hasattr(content, "__iter__"):
                    # Is a single grid
                    content = [content]
                for agent in content:
                    data = agent_portrayal(agent)
                    x.append(i)
                    y.append(j)
                    if "size" in data:
                        s.append(data["size"])
                    if "color" in data:
                        c.append(data["color"])
        out = {"x": x, "y": y}
        # This is the default value for the marker size, which auto-scales
        # according to the grid area.
        out["s"] = (180 / max(g.width, g.height)) ** 2
        if len(s) > 0:
            out["s"] = s
        if len(c) > 0:
            out["c"] = c
        return out

    space_ax.set_xlim(-1, space.width)
    space_ax.set_ylim(-1, space.height)
    space_ax.scatter(**portray(space))

In [15]:
def agent_portrayal(agent):
    size = 10
    color = "tab:red"
    if agent.wealth > 0:
        size = 50
        color = "tab:blue"
    return {"size": size, "color": color}

In [16]:
space_fig = Figure()
space_ax = space_fig.subplots()
space = getattr(model, "grid", None)

In [18]:
_draw_grid(space, space_ax, agent_portrayal)
solara.FigureMatplotlib(space_fig, format="png")

Cannot show ipywidgets in text

In [19]:
from mesa.agent import Agent
from cacheable_model import AgentSerializer
model = model
agents = [
    Agent(unique_id=1, model=model),
    Agent(unique_id=2, model=model)
]

# Serialize and save agents to a Parquet file
AgentSerializer.save_agents_to_parquet(agents, 'agents.parquet')

# Load agents from a Parquet file
loaded_agents = AgentSerializer.load_agents_from_parquet('agents.parquet', model)

In [20]:
loaded_agents

[<mesa.agent.Agent at 0x7f77b5ee28f0>, <mesa.agent.Agent at 0x7f77b5ee33a0>]

In [21]:
import networkx as nx
import solara
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator

In [22]:
space


In [23]:
space.__dict__

{'height': 10,
 'width': 10,
 'torus': True,
 'num_cells': 100,
 '_grid': [[[<__main__.MoneyAgent at 0x7f77e00c5480>],
   [],
   [],
   [],
   [],
   []],
  [[],
   [],
   [],
   [],
   [],
  [[],
   [],
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f77e00c6020>,
   [],
   [],
   [],
   []],
  [[<__main__.MoneyAgent at 0x7f77e00c52a0>,
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f77e00c4a60>],
   [],
   [],
   [],
   [],
  [[],
   [],
   [],
   [],
  [[],
   [],
  [[],
   [],
   [],
  [[],
   [],
   [],
   [],
   [],
 '_empties_built': False,
 '_neighborhood_cache': {((3, 4), True, False, 1): ((2, 3),
   (2, 4),
   (2, 5),
   (3, 3),
   (3, 5),
   (4, 3),
   (4, 4),
   (4, 5)),
  ((6, 1), True, False, 1): ((5, 0),
   (5, 1),
   (5, 2),
   (6, 0),
   (6, 2),
   (7, 0),
   (7, 1),
   (7, 2)),
  ((7, 7), True, False, 1): ((6, 6),
   (6, 7),
   (6, 8),
   (7, 6),
   (7, 8),
   (8, 6),
   (8, 7),
   (8, 8)),
  ((8, 0), True, False, 1): ((7, 9),
   (7, 0),
   (7, 1),
   (8, 9),
   (8, 1),
  

In [42]:
import pandas as pd

def cache_grid_state(grid, filename='grid_cache.parquet'):
    grid_state = {
        'width': grid.width,
        'height': grid.height,
        'agents': []
    }
    for x in range(grid.width):
        for y in range(grid.height):
            cell_contents = grid._grid[x][y]
            if cell_contents:
                if not hasattr(cell_contents, "__iter__"):
                    cell_contents = [cell_contents]
                for agent in cell_contents:
                    print(agent.__dict__)
                    print(f'{type(agent)=}')
                    agent_state = {
                        'pos_x': agent.pos[0],
                        'pos_y': agent.pos[1],
                        'unique_id': agent.unique_id,
                        'wealth': agent.wealth,
                        # **agent.__dict__
                    }
                    grid_state['agents'].append(agent_state)

    # Convert to DataFrame
    df = pd.DataFrame(grid_state['agents'])
    print(df)
    # Save DataFrame to Parquet
    df.to_parquet(filename)

cache_grid_state(model.grid)

{'unique_id': 73, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 0), 'wealth': 2}
type(agent)=<class '__main__.MoneyAgent'>
{'unique_id': 43, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 1), 'wealth': 0}
type(agent)=<class '__main__.MoneyAgent'>
{'unique_id': 19, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 1), 'wealth': 0}
type(agent)=<class '__main__.MoneyAgent'>
{'unique_id': 58, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 1), 'wealth': 0}
type(agent)=<class '__main__.MoneyAgent'>
{'unique_id': 10, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 2), 'wealth': 3}
type(agent)=<class '__main__.MoneyAgent'>
{'unique_id': 29, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 5), 'wealth': 0}
type(agent)=<class '__main__.MoneyAgent'>
{'unique_id': 4, 'model': <__main__.MoneyModel object at 0x7f77e00c45e0>, 'pos': (0, 5), 'wealth': 0}
type(agent)=<class '__main__

In [47]:
table = pd.read_parquet("grid_cache.parquet")
table

,pos_x,pos_y,unique_id,wealth
0,0,0,73,2
1,0,1,43,0
2,0,1,19,0
3,0,1,58,0
4,0,2,10,3
...,...,...,...,...
95,9,6,1,0
96,9,6,15,0
97,9,7,35,4
98,9,7,56,1


In [25]:
# not implemented fully

import pandas as pd
from mesa.space import MultiGrid as Grid
# import mesa.space.MultiGrid as Grid
def reconstruct_grid(filename='grid_cache.parquet'):
    # Load the DataFrame from Parquet
    df = pd.read_parquet(filename)

    # Create a new Grid instance
    width = df['pos_x'].max() + 1  # Assuming positions start from 0
    height = df['pos_y'].max() + 1  # Assuming positions start from 0
    grid = Grid(width, height, False)

    # Add agents to the grid
    for _, row in df.iterrows():
        agent = Agent(pos=(row['pos_x'], row['pos_y']), wealth=row['wealth'])
        grid.add_agent(agent, row['pos_x'], row['pos_y'])

    return grid

# Usage
model_grid = reconstruct_grid('grid_cache.parquet')
print(model_grid)

TypeError: Agent.__init__() got an unexpected keyword argument 'pos'

# IGNORE BELOW KIV

In [ ]:
def agent_portrayal(agent):
    size = 10
    color = "tab:red"
    if agent.wealth > 0:
        size = 50
        color = "tab:blue"
    return {"size": size, "color": color}

class dummy:

    def __init__(self):


def _draw_grid(space, space_ax, agent_portrayal):
    def portray(g):
        x = []
        y = []
        s = []  # size
        c = []  # color
        for i in range(g.width):
            for j in range(g.height):
                content = g._grid[i][j]
                if not content:
                    continue
                if not hasattr(content, "__iter__"):
                    # Is a single grid
                    content = [content]
                for agent in content:
                    data = agent_portrayal(agent)
                    x.append(i)
                    y.append(j)
                    if "size" in data:
                        s.append(data["size"])
                    if "color" in data:
                        c.append(data["color"])
        out = {"x": x, "y": y}
        # This is the default value for the marker size, which auto-scales
        # according to the grid area.
        out["s"] = (180 / max(g.width, g.height)) ** 2
        if len(s) > 0:
            out["s"] = s
        if len(c) > 0:
            out["c"] = c
        return out

    space_ax.set_xlim(-1, space.width)
    space_ax.set_ylim(-1, space.height)
    space_ax.scatter(**portray(space))

In [ ]:
grid_state = {}
grid_state["height"] = space.height
grid_state["width"] = space.width
grid_state["torus"] = space.torus
grid_state["num_cells"] = space.num_cells
# grid_state["_grid"] = [
#     [self.default_val() for _ in range(self.height)] for _ in range(self.width)
# ]
for i in range(space.height):
    for j in range(space.width):
        for agent_index in range(len(space._grid[i][j])):
            agent = space._grid[i][j][agent_index]
            agent_dict = {}
            agent_dict["unique_id"] = agent.unique_id
            agent_dict["pos"] = agent.pos
            agent_dict["wealth"] = agent.wealth
            space._grid[i][j][agent_index] = agent_dict


In [ ]:
space.__dict__

In [ ]:
df = pd.DataFrame(space.__dict__)

In [ ]:
space.__dict__.keys()

In [ ]:
_draw_grid(space, space_ax, agent_portrayal)
solara.FigureMatplotlib(space_fig, format="png")

In [ ]:
model._steps

In [ ]:
_draw_grid(space, space_ax, agent_portrayal)
solara.FigureMatplotlib(space_fig, format="png")

In [ ]:
import pandas as pd
from typing import List, Any

def read_grid_state(filename='grid_cache.parquet', model: Any = None) -> Any:
    """Read the grid state from a Parquet file and reconstruct the grid with agents."""
    # Load DataFrame from Parquet
    df = pd.read_parquet(filename)

    # Extract grid dimensions from the file
    width = int(df['width'].iloc[0])
    height = int(df['height'].iloc[0])

    # Initialize the grid
    grid = Grid(width=width, height=height)  # Adjust as needed for your grid class

    # Reconstruct agents and place them in the grid
    agents = df[['pos_x', 'pos_y', 'wealth', 'unique_id']]
    for _, row in agents.iterrows():
        pos_x = int(row['pos_x'])
        pos_y = int(row['pos_y'])
        unique_id = int(row['unique_id'])
        wealth = float(row['wealth'])

        # Create agent instance (adjust based on how you create agents)
        agent = Agent(unique_id=unique_id, model=model)
        agent.pos = (pos_x, pos_y)
        agent.wealth = wealth

        # Place agent in the grid
        grid._grid[pos_x][pos_y] = grid._grid[pos_x][pos_y] or []
        grid._grid[pos_x][pos_y].append(agent)

    return grid


In [ ]:
# Example of loading a grid from a Parquet file
  # Your model instance, or a placeholder if required
grid = read_grid_state('grid_cache.parquet', model=model)


In [ ]:

gini = model.datacollector.get_model_vars_dataframe()
# Plot the Gini coefficient over time
g = sns.lineplot(data=gini)
g.set(title="Gini Coefficient over Time", ylabel="Gini Coefficient");

In [ ]:
gini

In [ ]:
agent_wealth

In [ ]:
agent_wealth = model.datacollector.get_agent_vars_dataframe().sort_index()
agent_wealth


In [ ]:
agent_wealth.iloc[:,0]

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import glob

# Get a list of all Parquet files
model_files = glob.glob('test_path/model_data_*.parquet')
agent_files = glob.glob('test_path/agent_data_*.parquet')

# Initialize lists to hold dataframes
model_dfs = []
agent_dfs = []

# Read and append each file to the list
for model_file in model_files:
    table = pq.read_table(model_file)
    df = table.to_pandas()
    model_dfs.append(df)

for agent_file in agent_files:
    table = pq.read_table(agent_file)
    df = table.to_pandas()
    agent_dfs.append(df)
    print(agent_dfs)
    print("----")

# Concatenate all DataFrames
model_df = pd.concat(model_dfs, ignore_index=True)
agent_df = pd.concat(agent_dfs)

# Display the combined DataFrames


In [ ]:
model_df
g = sns.lineplot(data=model_df)
g.set(title="Gini Coefficient over Time", ylabel="Gini Coefficient");

In [ ]:
model_df

In [ ]:
agent_df

In [ ]:
model_df

In [ ]:
(gini == model_df).sum()


In [ ]:
table = pq.read_table("test_path/special_results.parquet")
df = table.to_pandas()
df

In [ ]:


# Read and combine the data
model_df, agent_df = cacheable_model.combine_dataframes()

# Display the combined DataFrames
model_df
agent_df
